<a href="https://colab.research.google.com/github/JunSeokCheon/Natural-Language-based-Climate-Technology-Classification/blob/master/RandomForest(Okt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install konlpy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
cd /content/drive/MyDrive/dacon

In [ ]:
train=pd.read_csv('preprocessing_train.csv')
test=pd.read_csv('preprocessing_test.csv')
sample_submission=pd.read_csv('sample_submission.csv')

In [ ]:
train.head(20)

In [ ]:
test.head(2)

In [ ]:
sample_submission.head(6)

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
5,174309,0


In [ ]:
#데이터 구조 파악
print(train.shape)
print(test.shape)
print(sample_submission.shape)

In [ ]:
#심각한 불균형 데이터임을 알 수 있습니다.
train.label.value_counts(sort=False)/len(train)

In [ ]:
# train = train[['과제명', '요약문_한글키워드', '요약문_영문키워드','label']]
# test = test[['과제명', '요약문_한글키워드', '요약문_영문키워드']]
# train[['과제명', '요약문_한글키워드', '요약문_영문키워드', 'label']].fillna('', inplace=True)
# test[['과제명', '요약문_한글키워드', '요약문_영문키워드']].fillna('', inplace=True)

In [ ]:
train = train[['요약문_영문키워드', 'label']]
test = test[['요약문_영문키워드']]
train.fillna('', inplace=True)
test.fillna('', inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
# train['data'] = train['과제명'] + "\n" + train['요약문_한글키워드'] + "\n" + train['요약문_영문키워드']
# test['data'] = test['과제명'] + "\n" + test['요약문_한글키워드'] + "\n" + train['요약문_영문키워드']

In [ ]:
train['data'] = train['요약문_영문키워드']
test['data'] = test['요약문_영문키워드'] 

In [ ]:
train['data']

In [ ]:
#1. re.sub 한글 및 공백을 제외한 문자 제거
#2. okt 객체를 활용해 형태소 단위로 나눔
#3. remove_stopwords로 불용어 제거 
def preprocessing(text, okt, remove_stopwords=False):
    # text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
    word_text=okt.morphs(text, stem=True)
    if remove_stopwords:
        word_review=[token for token in word_text if not len(token)==1]
    return word_review

In [ ]:
# stop_words=['은','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한']
okt=Okt()
clean_train_text=[]
clean_test_text=[]

In [ ]:
#시간이 많이 걸립니다.
for text in tqdm.tqdm(train['data']):
    try:
        clean_train_text.append(preprocessing(text, okt, remove_stopwords=True))
    except:
        clean_train_text.append([])

In [ ]:
for text in tqdm.tqdm(test['data']):
    if type(text) == str:
        clean_test_text.append(preprocessing(text, okt, remove_stopwords=True))
    else:
        clean_test_text.append([])

In [ ]:
len(clean_train_text)

In [ ]:
clean_train_text[15]

In [ ]:
len(clean_test_text)

In [ ]:
clean_test_text[5]

In [ ]:
#추후 재사용 가능하도록 npy로 전환
DATA_IN_PATH='./data_in/'
TRAIN_INPUT_DATA = 'train_ENkey.npy'
TEST_INPUT_DATA = 'test_ENkey.npy'

import os
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)
    
np.save(open(DATA_IN_PATH+TRAIN_INPUT_DATA, 'wb'), clean_train_text)
np.save(open(DATA_IN_PATH+TEST_INPUT_DATA, 'wb'), clean_test_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#tokenizer 인자에는 list를 받아서 그대로 내보내는 함수를 넣어줍니다. 또한 소문자화를 하지 않도록 설정해야 에러가 나지 않습니다.
vectorizer = CountVectorizer(tokenizer = lambda x: x, lowercase=False)
train_features=vectorizer.fit_transform(clean_train_text)
test_features=vectorizer.transform(clean_test_text)
#test데이터에 fit_transform을 할 경우 data leakage에 해당합니다
#https://deepinsight.tistory.com/165

In [ ]:
train_features

In [ ]:
test_features

In [ ]:
#훈련 데이터 셋과 검증 데이터 셋으로 분리
TEST_SIZE=0.2
RANDOM_SEED=42

train_x, eval_x, train_y, eval_y=train_test_split(train_features, train['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [ ]:
len(eval_y)

In [ ]:
#랜덤포레스트로 모델링
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(n_estimators=100)

forest.fit(train_x, train_y)

In [ ]:
#모델 검증
forest.score(eval_x, eval_y)

In [ ]:
forest.predict(test_features)

In [ ]:
sample_submission['label']=forest.predict(test_features)

In [ ]:
sample_submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,0
43574,217878,0


In [ ]:
sample_submission.to_csv('train_ENkey.csv', index=False)